# Проект 6. Рекомендательные системы.

# Часть 2: обучение моделей, улучшение их, выводы

In [1]:
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from sklearn.model_selection import train_test_split
import sklearn
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
import warnings

import pandas as pd
import numpy as np
import scipy
from IPython.core.display import display
from tqdm import tqdm
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable
%matplotlib inline
warnings.filterwarnings('ignore')

/Users/alexeizh/Alexeizh_DST/venv/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Загрузим подготовленные в другом ноутбуке данные. Это тренировочная база, в которую добавлены признаки, числовые признаки
стандартизированы, категориальные переведены в dummies.

Идея такова - сначала обучим "базовую" модель и получим на ней метрики. *В базовую модель сразу внесу изменения,
полученные опытным путем в черновике - lerning rate = 0.08 (вместо 0.1) и NUM_COMPONENTS=35 (а не 30, как в оригинале)*

Затем будем постепенно добавлять фичи и смотреть насколько это улучшает или ухудшает предсказания.

In [3]:
# загрузим тренировочную базу
train = pd.read_csv(
    '/Users/alexeizh/Alexeizh_DST/Unit_7/final_prjt/train_for_model.csv')
train.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
# разобьем на тестовую и тренинговую
train_data, test_data = train_test_split(train, random_state=32, shuffle=True)

In [5]:
# создаем разреженную матрицу
total_users = max(train_data['userid'].max(), test_data['userid'].max()) + 1
total_items = max(train_data['itemid'].max(), test_data['itemid'].max()) + 1
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])), shape=(total_users, total_items))

In [23]:
NUM_THREADS = 4  # число потоков
NUM_COMPONENTS = 35  # число параметров вектора
NUM_EPOCHS = 20  # число эпох обучения

model = LightFM(learning_rate=0.08, loss='logistic',
                no_components=NUM_COMPONENTS)
model_base = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                       num_threads=NUM_THREADS)

In [24]:
preds_base = model_base.predict(test_data.userid.values,
                                test_data.itemid.values)

In [25]:
sklearn.metrics.roc_auc_score(test_data.rating, preds_base)

0.7478793066608688

Итак, ROC_AUC базовой модели составляет **0.7478793066608688**

### Начнем добавлять (постепенно) фичи и смотреть как это повлияет на результат

Для начала сделаем базу, из которой будем доставать фичи для обучения и тестов. _То есть сгруппируем всю имеющуюся базу
по itemid и потом постепенно будем вынимать из нее фичи для дальнейшего обучения и тестов (сразу уберу из базы все признаки,
касающиеся времени покупки - они бесползены, как показали опыты на "черновиках")_

In [13]:
feat_data = train.groupby('itemid')[['verified', 'desc_subj',
                                     'vote', 'numb_of_cat', 'rank_in_main', 'price', 'main_cat_All Beauty',
                                     'main_cat_Amazon Home', 'main_cat_Arts, Crafts & Sewing',
                                     'main_cat_Baby', 'main_cat_Camera & Photo',
                                     'main_cat_Cell Phones & Accessories', 'main_cat_Grocery',
                                     'main_cat_Grocery & Gourmet Food', 'main_cat_Health & Personal Care',
                                     'main_cat_Home Audio & Theater', 'main_cat_Industrial & Scientific',
                                     'main_cat_Musical Instruments', 'main_cat_Office Products',
                                     'main_cat_Pet Supplies', 'main_cat_Software',
                                     'main_cat_Sports & Outdoors', 'main_cat_Tools & Home Improvement',
                                     'main_cat_Toys & Games']].mean().copy()

In [17]:
# теперь сделаем реиндексацию, чтобы учесть все возможные itemid
new_indexies = range(total_items)
feat_data = feat_data.reindex(new_indexies)

In [18]:
# Посмотрим, не появились ли пропуски
feat_data.isna().sum()

verified                              18
desc_subj                             18
vote                                  18
numb_of_cat                           18
rank_in_main                          18
price                                 18
main_cat_All Beauty                   18
main_cat_Amazon Home                  18
main_cat_Arts, Crafts & Sewing        18
main_cat_Baby                         18
main_cat_Camera & Photo               18
main_cat_Cell Phones & Accessories    18
main_cat_Grocery                      18
main_cat_Grocery & Gourmet Food       18
main_cat_Health & Personal Care       18
main_cat_Home Audio & Theater         18
main_cat_Industrial & Scientific      18
main_cat_Musical Instruments          18
main_cat_Office Products              18
main_cat_Pet Supplies                 18
main_cat_Software                     18
main_cat_Sports & Outdoors            18
main_cat_Tools & Home Improvement     18
main_cat_Toys & Games                 18
dtype: int64

In [19]:
# Заполним все пропуски нулями
feat_data.fillna(0, inplace=True)

In [20]:
# Зададим так же матрицу идентичности
identity_matrix = sparse.identity(total_items)

Теперь можем приступить к постепенному добавлению фичей и сравнению результатов.
Пойдем от меньшего к большему - то есть сначала будем добавлять по одной фиче и смотреть, потом выберем
самую хорошую и будем добавлять к ней дополнения.

Начнем с **price**

In [21]:
# сначала зададим разреженную матрицу этого признака
price_сsr = csr_matrix(feat_data[['price']].astype(int))

In [22]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
price_feat = sparse.hstack([identity_matrix, price_сsr])

In [26]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_price = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                        num_threads=NUM_THREADS, item_features=price_feat)

preds_price = model_price.predict(test_data.userid.values,
                                  test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_price)

0.7475614642320709

Итак, ROC_AUC с фичей **price** составляет **0.7475614642320709**, что немного хуже чем базовая

Посмотрим что будет, если использовать __rank_in_main__

In [27]:
# сначала зададим разреженную матрицу этого признака
rank_сsr = csr_matrix(feat_data[['rank_in_main']].astype(int))

In [28]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
rank_feat = sparse.hstack([identity_matrix, rank_сsr])

In [29]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_rank = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                       num_threads=NUM_THREADS, item_features=rank_feat)

preds_rank = model_rank.predict(test_data.userid.values,
                                test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_rank)

0.7474850350098888

Итак, ROC_AUC с фичей **rank_in_main** составляет **0.7474850350098888**, что немного хуже чем базовая
и чем модель с ценой

Посмотрим что будет, если использовать __desc_subj__ _(то есть объективность/субъективность описания товара)_

In [ ]:
# сначала зададим разреженную матрицу этого признака
subj_сsr = csr_matrix(feat_data[['desc_subj']].astype(int))

In [ ]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
subj_feat = sparse.hstack([identity_matrix, subj_сsr])

In [ ]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_subj = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                       num_threads=NUM_THREADS, item_features=subj_feat)

preds_subj = model_subj.predict(test_data.userid.values,
                                test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_subj)

Итак, ROC_AUC с фичей **desc_subj** составляет **0.7326653244275896**, что заметно хуже чем базовая
и чем предыдущие фичи

Посмотрим что будет, если использовать __vote__ _(то есть количество проголосовавших за отзыв о товаре людей)_

In [ ]:
# сначала зададим разреженную матрицу этого признака
vote_сsr = csr_matrix(feat_data[['vote']].astype(int))

In [ ]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
vote_feat = sparse.hstack([identity_matrix,vote_сsr])

In [ ]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_vote = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                       num_threads=NUM_THREADS, item_features=vote_feat)

preds_vote = model_vote.predict(test_data.userid.values,
                                test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_subj)

Итак, ROC_AUC с фичей **vote** составляет **0.7326653244275896**, что заметно хуже чем базовая
и чем предыдущие фичи

Посмотрим что будет, если использовать __numb_of_cat__ _(то есть количество категорий, в которых представлен товар)_

In [36]:
# сначала зададим разреженную матрицу этого признака
cat_num_сsr = csr_matrix(feat_data[['numb_of_cat']].astype(int))

In [37]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
cat_num_feat = sparse.hstack([identity_matrix, cat_num_сsr])

In [41]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_cat_num = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                          num_threads=NUM_THREADS, item_features=cat_num_feat)

preds_cat_num = model_cat_num.predict(test_data.userid.values,
                                      test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_cat_num)

0.7471678192483783

Итак, ROC_AUC с фичей **numb_of_cat** составляет **0.7471678192483783**, почти аналогично базовой модели

Посмотрим что будет, если в качестве фичи использовать dummies категорий

In [39]:
# сначала зададим разреженную матрицу этого признака
cat_сsr = csr_matrix(feat_data[['main_cat_All Beauty',
                                'main_cat_Amazon Home', 'main_cat_Arts, Crafts & Sewing',
                                'main_cat_Baby', 'main_cat_Camera & Photo',
                                'main_cat_Cell Phones & Accessories', 'main_cat_Grocery',
                                'main_cat_Grocery & Gourmet Food', 'main_cat_Health & Personal Care',
                                'main_cat_Home Audio & Theater', 'main_cat_Industrial & Scientific',
                                'main_cat_Musical Instruments', 'main_cat_Office Products',
                                'main_cat_Pet Supplies', 'main_cat_Software',
                                'main_cat_Sports & Outdoors', 'main_cat_Tools & Home Improvement',
                                'main_cat_Toys & Games']].astype(int))

In [40]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
cat_feat = sparse.hstack([identity_matrix, cat_сsr])

In [42]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_cat = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                      num_threads=NUM_THREADS, item_features=cat_feat)

preds_cat = model_cat.predict(test_data.userid.values,
                              test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_cat)

0.691614962452294

Итак, ROC_AUC с фичей **dummies** составляет **0.691614962452294**, эта фича работает хуже всего.

Посмотрим что будет, если в качестве фичи использовать сочетание трех более или менее удачных фичей:
цена, рейтинг в основной категории и количество категорий, в которы представлен товар

In [44]:
# сначала зададим разреженную матрицу этого признака
best_сsr = csr_matrix(
    feat_data[['price', 'rank_in_main', 'numb_of_cat']].astype(int))

In [45]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
best_feat = sparse.hstack([identity_matrix, best_сsr])

In [46]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_best = model.fit(ratings_coo, epochs=NUM_EPOCHS,
                       num_threads=NUM_THREADS, item_features=best_feat)

preds_best = model_cat.predict(test_data.userid.values,
                               test_data.itemid.values)

sklearn.metrics.roc_auc_score(test_data.rating, preds_best)

0.7477287381309115

Итак, ROC_AUC с тремя фичами составляет **0.7477287381309115**, все равно чуть хуже базовой модели.

Попробуем сделать предсказния для тестовой базы kaggle на базовой и базовой с фичами модели.

In [47]:
# загружаем базу
test = pd.read_csv('test_new.csv')

Признаки нужно нормализовать. Вернее нормализуем только **price, rank_in_main и numb_of_cat**,
так как сейчас будем использовать только их

In [49]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()

# отберем признаки для нормализации
test_for_scaler = test[['price', 'rank_in_main', 'numb_of_cat']]

In [50]:
test_for_scaler = scaler.fit_transform(test_for_scaler)

test_scaled = pd.DataFrame(test_for_scaler, columns=[
                           'price', 'rank_in_main', 'numb_of_cat'])

In [52]:
test['price'] = test_scaled['price']
test['rank_in_main'] = test_scaled['rank_in_main']
test['numb_of_cat'] = test_scaled['numb_of_cat']

In [55]:
test.drop(['Unnamed: 0'], axis=1, inplace=True)

Теперь обучаем модель и получаем предсказания для базовой версии и для версии с тремя фичами


In [56]:
# создаем новую разреженную матрицу
total_users = max(train['userid'].max(), test['userid'].max()) + 1
total_items = max(train['itemid'].max(), test['itemid'].max()) + 1
ratings_coo_sub = sparse.coo_matrix((train['rating'].astype(int),
                                     (train['userid'],
                                      train['itemid'])), shape=(total_users, total_items))

In [57]:
# используем модель с измененными настройками
NUM_THREADS = 4  # число потоков
NUM_COMPONENTS = 35  # число параметров вектора
NUM_EPOCHS = 20  # число эпох обучения

# обучаем базовую модель
model = LightFM(learning_rate=0.08, loss='logistic',
                no_components=NUM_COMPONENTS)
model_base_tuned = model.fit(ratings_coo_sub, epochs=NUM_EPOCHS,
                             num_threads=NUM_THREADS)

# получаем предсказания на базовой модели
preds_tuned = model_base_tuned.predict(test.userid.values,
                                       test.itemid.values)

# нормализуем предсказания
normalized_preds_tuned = (preds_tuned - preds_tuned.min()) / \
    (preds_tuned - preds_tuned.min()).max()
normalized_preds_tuned.min(), normalized_preds_tuned.max()

# подгрузим шаблон для submission
submission = pd.read_csv(
    '/Users/alexeizh/Alexeizh_DST/Unit_7/final_prjt/final_hakaton/recommendationsv4/sample_submission.csv')

# занесем предсказания базовой модели в шаблон
submission['rating'] = normalized_preds_tuned

# записываем в файл для публикации
submission.to_csv('submission_base.csv', index=False)

Теперь сделаем обучение и предсказание на модели с тремя фичами

In [61]:
# Для начала создадим базу из которой будем выбирать фичи.
# Объединим train и test (но только столбцы для обучения
data_feat_1 = train[['itemid', 'price', 'rank_in_main', 'numb_of_cat']].copy()
data_feat_2 = test[['itemid', 'price', 'rank_in_main', 'numb_of_cat']].copy()
data_feat_final = pd.concat([data_feat_1, data_feat_2], axis=0)

In [64]:
# теперь группируем по itemid для получения нужных фичей и последующего создания матрицы фичей
feat_data_final = data_feat_final.groupby(
    'itemid')[['price', 'rank_in_main', 'numb_of_cat']].mean().copy()

In [65]:
# теперь сделаем реиндексацию, чтобы учесть все возможные itemid
new_indexies_final = range(total_items)
feat_data_final = feat_data_final.reindex(new_indexies_final)

In [66]:
# Посмотрим, не появились ли пропуски
feat_data_final.isna().sum()
# пропусков нет

price           0
rank_in_main    0
numb_of_cat     0
dtype: int64

In [67]:
# Зададим так же новую матрицу идентичности
identity_matrix_final = sparse.identity(total_items)

In [68]:
# зададим новую разреженную матрицу признаков
best_сsr_final = csr_matrix(
    feat_data_final[['price', 'rank_in_main', 'numb_of_cat']].astype(int))

In [69]:
# теперь соединим ее с матрицей идентичности, чтобы получить фичу для модели
best_feat_final = sparse.hstack([identity_matrix_final, best_сsr_final])

In [70]:
# теперь обучим модель, сделаем предсказание и посмотрим на изменение метрики
model_final = model.fit(ratings_coo_sub, epochs=NUM_EPOCHS,
                        num_threads=NUM_THREADS, item_features=best_feat_final)

preds_final = model_cat.predict(test.userid.values,
                                test.itemid.values)

# нормализуем предсказания
normalized_preds_final = (preds_final - preds_final.min()) / \
    (preds_final - preds_final.min()).max()
normalized_preds_final.min(), normalized_preds_final.max()

# занесем предсказания базовой модели в шаблон
submission['rating'] = normalized_preds_final

# записываем в файл для публикации
submission.to_csv('submission_final.csv', index=False)

Время сделать сабмит на Каггл и посмотреть на результаты.
Сабмит с базовой моделью - submission_base - 0.76806 (19 место)
Сабмит с фичами - submission_final - 0.74705


__Увы, но использование полученных фичей не улучшило результат работы модели,
поэтому лучше остановиться на базовой модели с измененными настройками.__

### Займемся эмбеддингами

In [ ]:
item_biases, item_embeddings = model_base_tuned.get_item_representations()

In [ ]:
import nmslib

# Создаём наш граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')

# Начинаем добавлять наши товары в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [ ]:
# Вспомогательная функция для поиска по графу
def nearest_item_nms(itemid, index, n=10):
    nn = index.knnQuery(item_embeddings[itemid], k=n)
    return nn

In [ ]:
# Попробуем найти товары, похожие на товар с неким itemid
nbm = nearest_item_nms(37138, nms_idx)[0]
nbm

In [ ]:
# запишем полученные эмбеддинги в файл для использования в другом ноутбуке
import pickle

In [ ]:
with open('embeddings.pickle', 'wb') as f:
    pickle.dump(item_embeddings, f)

## Финальные выводы

1. Активность покупателей в последнее время падает. Пик покупательской активности приходился на период 5-6 летней давности,
с тех пор активность ощутимо уменьшается год от года.

2. Покупательская активность примерно одинакова в течение года, с небольшим уменьшением в летне-осенний период

3. Покупательсая активность в течение недели примерно одинакова, с небольшим падением активности в выходные дни

4. Описание товара, его "популярность" у других пользователей не оказывает существенного влияния на решение о покупке
(я считаю, что rating, показывающий "нравится или не нравится" покупателю товар связан с решением о покупке непосредственно)

5. При холодном старте, тем не менее, новому покупателю я бы предлагал товар с наибольшим overall в категории
(нужно "спросить" у покупателя товар какой категории он ищет). НО (!) этот показатель нужно модифицировать, учитывая голоса,
отданные за тот же товар (overall + overall*vote), в случае одинаковых значений этого модифицированного показателя для разных
товаров нужно предлагать тот, у кого выше эмоциональная оценка текста отзывов других пользователей.

6. Внедрение нашей модели позволит увеличить количество покупок, если мы будем предлагать покупателю товар,
который чаще всего покупают с уже купленным товаром (или в случае холодного старта - самые популярные товары
в интересующей покупателя категории).

7. В качестве метрики я бы испоьзовал полноту предсказаний. Так как предложить покупателю больше потенциально инетерсных
товаров выгоднее, чем предложить один "точно" подходящий.

8. Использование дополнительных фичей, которые можно было получить из имеющихся данных, только ухудшало работу модели,
поэтому пришлось от них отказаться.

9. К сожалению в предоставленных данных мало информации о собственно покупателе. Их использование могло бы (вероятно)
улучшить результаты работы модели

10. Был проведен анализ текстов отзывов покупателя, но использование этих данных (использовано в черновиках)
не принесло никакого положительного результата


